<a href="https://colab.research.google.com/github/mdehghani86/AppliedGenAI/blob/main/M10_2_Atlas_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="border: 2px solid #1f77b4; border-radius: 12px; padding: 20px; background-color: #f9fbff;">

# ATLAS AI Travel Assistant

<small><span style="color:#1f77b4;"><strong>Designed by Dr. Dehghani</strong></span></small>

---

## 🧭 Overview
<span style="color:#1f77b4;">LangGraph-powered travel assistant</span> that finds flights and creates itineraries through natural language.

---

## 🏗️ Architecture
<pre>
START → Supervisor (Menu)  
↓  
┌──────┼──────┐  
│      │      │  
TripInfo Flights Itinerary  
│      │      │  
└──────┼──────┘  
↓  
Supervisor → END
</pre>

---

## 🛠️ Key Features
- <span style="color:#1f77b4;"><strong>Smart Parsing</strong></span>: "NYC to Paris next week" → structured travel data  
- <span style="color:#1f77b4;"><strong>Date Intelligence</strong></span>: Converts "next week" to actual dates  
- <span style="color:#1f77b4;"><strong>Flight Search</strong></span>: Top 3 options + Google Flights booking links  
- <span style="color:#1f77b4;"><strong>Custom Itineraries</strong></span>: 6 trip types with feedback loop  
- <span style="color:#1f77b4;"><strong>Multilingual</strong></span>: Destination-based greetings via LLM

---

## 🧩 Core Nodes
1. <span style="color:#1f77b4;">Supervisor</span>: Central controller, menu routing  
2. <span style="color:#1f77b4;">Trip Info</span>: Natural language parsing, date conversion  
3. <span style="color:#1f77b4;">Flights</span>: 3 options with booking links  
4. <span style="color:#1f77b4;">Itinerary</span>: Customized plans based on trip type  

---

## 🧪 Usage
```python
show_graph()  # View workflow  
run_atlas()   # Start planning


In [ ]:
# 🔧 ATLAS AI Setup – Install Required Packages & Import Modules (Colab Ready)

# 📦 Install required libraries quietly
!pip install --quiet langgraph langchain-openai langchain-community tavily-python graphviz openai

# 📂 Standard libraries
import os, json                             # Env vars + JSON parsing
from typing import TypedDict, Literal       # Type-safe state definitions

# 🧠 LangChain + LangGraph
from langchain_openai import ChatOpenAI     # OpenAI LLM access
from langchain_core.messages import HumanMessage, SystemMessage  # Message format
from langgraph.graph import StateGraph, END                     # Graph engine

# 🔍 Tavily for real-time travel search
from tavily import TavilyClient


In [ ]:
# +++++ SECTION 1: SETUP & IMPORTS +++++
# This section handles all necessary imports and API key configuration for the ATLAS AI travel assistant.
# It sets up the OpenAI connection, LangGraph components, and display utilities needed for the application.

import os
from typing import TypedDict, Literal
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
from langgraph.graph import StateGraph, END
from IPython.display import Image, display, HTML
from datetime import datetime, timedelta

# API Key Setup - automatically loads from Google Colab secrets
try:
    from google.colab import userdata
    openai_key = userdata.get("OPENAI_API_KEY")
    if openai_key:
        os.environ["OPENAI_API_KEY"] = openai_key
        print("✅ OpenAI API Key loaded successfully.")
    else:
        print("❌ OpenAI API Key not found in Colab secrets.")
except Exception as e:
    print(f"🚫 Error loading API key: {e}")

# Initialize the Language Model with optimal settings for travel planning
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)  # Temperature 0.7 for creative but focused responses

# +++++ SECTION 2: UI & DISPLAY FUNCTIONS +++++
# This section defines the visual interface components that make the assistant user-friendly.
# It includes HTML-styled output boxes and input prompts with color coding for different types of content.

def pretty_print(text, title="🤖 Model Response", style="blue"):
    """
    Enhanced HTML display function for beautiful, readable output.

    Purpose: Creates styled boxes for different types of content
    - Blue boxes: AI responses, results, confirmations
    - Red boxes: Questions, prompts, missing information alerts

    Args:
        text: Content to display inside the styled box
        title: Header text shown at the top of the box
        style: "blue" for responses, "red" for questions/alerts
    """
    lines = text.strip().split('\n')
    # Check if content is a bulleted list for special formatting
    is_bulleted = all(line.strip().startswith(("-", "•", "*")) for line in lines if line.strip())

    if is_bulleted:
        # Convert bulleted text to proper HTML list format
        list_items = ''.join(f"<li>{line.lstrip('-•* ').strip()}</li>" for line in lines if line.strip())
        content_html = f"<ul style='margin-top: 6px;'>{list_items}</ul>"
    else:
        # Convert line breaks to HTML breaks for regular text
        content_html = text.replace("\n", "<br>")

    # Color scheme selection based on content type
    if style == "red":
        bg_color = "#fff5f5"      # Light red background for questions
        border_color = "#e53e3e"   # Red border for attention
        text_color = "#742a2a"     # Dark red text for readability
    else:  # default blue theme
        bg_color = "#f8f9fc"      # Light blue background for responses
        border_color = "#4285f4"   # Google blue border
        text_color = "#202124"     # Dark gray text for readability

    # Generate and display the styled HTML box
    display(HTML(f"""
    <div style="background-color:{bg_color}; border-left:5px solid {border_color};
                padding:16px; margin-top:16px; font-family:'Segoe UI', sans-serif;
                color:{text_color}; line-height:1.6;">
      <strong>{title}</strong><br><br>
      {content_html}
    </div>
    """))

def get_input(prompt: str) -> str:
    """
    User input function with visual styling.
    Displays questions in red-styled boxes to distinguish from AI responses.
    """
    pretty_print(prompt, "💬 Question", "red")
    return input("👤 You: ").strip()

# +++++ SECTION 3: STATE MANAGEMENT +++++
# This section defines the data structure that flows through the entire LangGraph workflow.
# The TravelState acts as a shared memory that all nodes can read from and write to,
# maintaining conversation context and user preferences throughout the interaction.

class TravelState(TypedDict):
    """
    Central state object that maintains all conversation data.
    This flows through every node in the graph, allowing information sharing.
    """
    user_input: str              # Raw user input about their trip
    departure_city: str          # Parsed departure location
    destination: str             # Parsed destination location
    dates: str                   # Parsed and converted travel dates
    travelers: str               # Number of travelers
    service_type: str           # Current service being requested (flights/itinerary/done)
    result: str                 # LLM-generated results (flights or itinerary)
    trip_info_complete: bool    # Flag to track if all basic info is collected
    trip_type: str              # Type of trip for itinerary customization

# +++++ SECTION 4: SUPERVISOR NODE - CENTRAL CONTROLLER +++++
# The supervisor is the brain of the system. It manages the conversation flow, handles missing information,
# provides multilingual greetings based on destination, and routes users to appropriate services.
# This node is visited multiple times during a session, acting as the main menu and coordinator.

def supervisor(state: TravelState) -> TravelState:
    """
    Central controller node that manages the entire conversation flow.

    Responsibilities:
    1. Welcome new users with multilingual greeting
    2. Detect and collect missing travel information
    3. Provide destination-specific greetings using LLM
    4. Present service menu and route to appropriate services
    5. Validate all required information before proceeding
    """

    # First-time welcome message - only shown on initial visit
    if not state.get("trip_info_complete"):
        welcome_msg = """🎯 ATLAS AI - Your Smart Travel Assistant!
Ready to plan your perfect trip! ✈️🗺️"""
        pretty_print(welcome_msg, "🌍 Welcome to ATLAS AI")

    # Check if basic trip information collection is needed
    if not state.get("trip_info_complete"):
        state["service_type"] = "get_trip_info"  # Route to info collection first
        return state

    # Validate all required fields - critical for preventing errors in downstream services
    required_fields = {
        'departure_city': 'Which city are you departing from?',
        'destination': 'Where do you want to go?',
        'dates': 'When do you want to travel? (e.g., next week, July 15):',
        'travelers': 'How many travelers?'
    }

    # Check for missing information and collect it immediately
    missing_info = False
    for field, question in required_fields.items():
        if not state.get(field) or state.get(field) == "":
            pretty_print(f"❌ Missing: {field.replace('_', ' ').title()}", "⚠️ Missing Information", "red")
            state[field] = get_input(question)
            missing_info = True

    # Display confirmation when information is collected
    if missing_info:
        info_display = f"""✅ All information collected!

📍 FROM: {state['departure_city']}
📍 TO: {state['destination']}
📅 DATES: {state['dates']}
👥 TRAVELERS: {state['travelers']}"""
        pretty_print(info_display, "📋 Travel Information Confirmed")

    # Generate personalized greeting based on destination using LLM
    destination = state.get('destination', '')
    if destination:  # Only generate greeting if destination is available
        greeting_prompt = f"""Generate a brief welcome message for a traveler going to {destination}.
        Include a local greeting in the destination's language with translation.
        Keep it warm and encouraging, maximum 2 lines.

        Format: [Local greeting]! ([Translation])
        [Brief positive comment about the destination]"""

        try:
            # Get LLM-generated personalized greeting
            greeting_response = llm.invoke([SystemMessage(content=greeting_prompt)])
            pretty_print(greeting_response.content, "🌍 Personalized Welcome")
        except Exception as e:
            # Fallback if LLM call fails
            pretty_print(f"Welcome! {destination} is an amazing destination!", "🌍 Welcome")

    # Display current trip summary for user confirmation
    trip_summary = f"Your amazing trip: {state['departure_city']} → {state['destination']}"
    pretty_print(trip_summary, "🎯 Trip Summary")

    # Present main service menu with clear options
    menu_options = """What would you like me to help you with?

1. ✈️  Find Flights
2. 🗺️  Create Itinerary
3. Done"""

    pretty_print(menu_options, "🎯 Service Menu", "red")
    choice = get_input("Choose (1, 2, or 3):")

    # Route to appropriate service based on user choice
    if choice in ['1', 'flights', 'flight']:
        state["service_type"] = "flights"
        print("✈️ Searching for flights...")
    elif choice in ['2', 'itinerary', 'plan']:
        state["service_type"] = "itinerary"
        print("🗺️ Creating itinerary...")
    else:
        state["service_type"] = "done"  # This will trigger graph termination
        print("✅ Thank you for using ATLAS AI!")

    return state

# +++++ SECTION 5: TRIP INFORMATION COLLECTION +++++
# This node handles the intelligent parsing of user's travel plans using LLM capabilities.
# It can understand natural language input and convert relative dates like "next week" into specific dates.
# This is crucial for providing accurate flight and itinerary recommendations.

def get_trip_info(state: TravelState) -> TravelState:
    """
    Intelligent trip information parser using LLM for natural language understanding.

    Key Features:
    1. Parses natural language travel descriptions
    2. Converts relative dates ("next week") to actual dates
    3. Handles multiple city name formats and countries
    4. Validates and confirms all information before proceeding
    """

    print("📋 Let's gather your trip details...")

    # Get initial travel description from user
    if not state.get("user_input"):
        state["user_input"] = get_input("Tell me about your trip (where from, where to, when?):")

    # Advanced LLM prompt for parsing travel information with smart date conversion
    parse_prompt = f"""Extract and convert travel information from: "{state['user_input']}"

    IMPORTANT: Convert relative dates to actual dates based on today being {datetime.now().strftime('%B %d, %Y')}:
    - "next week" → actual week dates (e.g., "July 19-26, 2025")
    - "July" → "July 15, 2025"
    - "December" → "December 15, 2025"
    - "next month" → actual month name and year

    Return ONLY this exact format:
    FROM: [departure city]
    TO: [destination city]
    DATES: [converted actual dates - if unclear, return "NEEDS_DATES"]
    TRAVELERS: [number of people or "1 person"]

    Handle any global city names and convert all vague dates to specific ones."""

    try:
        # Use LLM to parse and convert the travel information
        response = llm.invoke([SystemMessage(content=parse_prompt)])
        response_text = response.content.strip()

        # Parse the structured LLM response into state variables
        for line in response_text.split('\n'):
            line = line.strip()
            if line.startswith('FROM:'):
                state["departure_city"] = line.replace('FROM:', '').strip()
            elif line.startswith('TO:'):
                state["destination"] = line.replace('TO:', '').strip()
            elif line.startswith('DATES:'):
                dates_value = line.replace('DATES:', '').strip()
                # Handle case where dates need clarification
                if dates_value == "NEEDS_DATES":
                    state["dates"] = ""
                else:
                    state["dates"] = dates_value
            elif line.startswith('TRAVELERS:'):
                state["travelers"] = line.replace('TRAVELERS:', '').strip()

        # Handle missing dates with additional user input and LLM conversion
        if not state.get("dates") or state["dates"] == "":
            date_info = """📅 I need specific travel dates for accurate results.

Examples that work well:
• 'next week' → I'll convert to actual dates
• 'July 15' → Specific date
• 'December 2025' → Month and year
• 'March 10-17' → Date range"""
            pretty_print(date_info, "📅 Date Information Needed", "red")
            user_date_input = get_input("When do you want to travel?")

            # Use LLM to convert user's date input to actual dates
            date_conversion_prompt = f"""Convert this date input to specific dates based on today being {datetime.now().strftime('%B %d, %Y')}:

            User input: "{user_date_input}"

            Examples of good conversions:
            - "next week" → "July 19-26, 2025" (actual week dates)
            - "July" → "July 15, 2025"
            - "December" → "December 15, 2025"
            - "next month" → actual month name and year

            Return only the converted date string, nothing else."""

            try:
                # Convert relative dates to specific dates
                date_response = llm.invoke([SystemMessage(content=date_conversion_prompt)])
                state["dates"] = date_response.content.strip()
            except:
                # Fallback to user input if conversion fails
                state["dates"] = user_date_input

        # Display comprehensive confirmation of all collected information
        info_display = f"""✅ TRIP INFORMATION CONFIRMED

📍 FROM: {state['departure_city']}
📍 TO: {state['destination']}
📅 DATES: {state['dates']}
👥 TRAVELERS: {state['travelers']}

✅ All information is complete and ready for flight/itinerary planning!"""
        pretty_print(info_display, "✅ Trip Information Confirmed")

        # Mark information collection as complete
        state["trip_info_complete"] = True

    except Exception as e:
        # Fallback manual collection if LLM parsing fails
        print(f"❌ Parsing error: {e}")
        print("Let me collect information manually...")

        # Manual information collection with date conversion
        state["departure_city"] = get_input("Which city are you departing from?")
        state["destination"] = get_input("Where do you want to go?")
        user_date_input = get_input("When do you want to travel?")

        # Still attempt LLM date conversion for manual input
        try:
            date_conversion_prompt = f"""Convert this to specific dates based on today being {datetime.now().strftime('%B %d, %Y')}:
            User input: "{user_date_input}"
            Return only the converted date string."""

            date_response = llm.invoke([SystemMessage(content=date_conversion_prompt)])
            state["dates"] = date_response.content.strip()
        except:
            state["dates"] = user_date_input

        state["travelers"] = get_input("How many travelers?") or "1 person"
        state["trip_info_complete"] = True

        # Display confirmation for manually collected information
        info_display = f"""✅ TRIP INFORMATION CONFIRMED

📍 FROM: {state['departure_city']}
📍 TO: {state['destination']}
📅 DATES: {state['dates']}
👥 TRAVELERS: {state['travelers']}

✅ All information collected successfully!"""
        pretty_print(info_display, "✅ Trip Information Confirmed")

    return state

# +++++ SECTION 6: FLIGHT SEARCH ENGINE +++++
# This node provides intelligent flight searching with realistic options and direct booking capabilities.
# It generates top flight recommendations and creates clickable Google Flights links with proper date formatting.
# The focus is on practical, actionable information that users can immediately use for booking.

def find_flights(state: TravelState) -> TravelState:
    """
    Advanced flight search engine that provides realistic flight options with booking capabilities.

    Features:
    1. Generates top 3 flight options with realistic pricing
    2. Includes detailed flight information (times, airports, stops)
    3. Creates functional Google Flights booking links
    4. Provides booking tips and price trend information
    5. User satisfaction feedback loop
    """

    print("✈️ Searching flight databases for best options...")

    # Optimized LLM prompt for concise but comprehensive flight information
    prompt = f"""Find TOP 3 realistic flight options for this route:

    Route: {state['departure_city']} → {state['destination']}
    Travel Date: {state['dates']}
    Passengers: {state['travelers']}

    Use this EXACT format (keep it concise):

    ✈️ OPTION 1: [Real Airline Name]
    💰 $[realistic price] per person
    ⏱️ [X]h [Y]m total travel time
    🛫 Depart [time] from [airport code]
    🛬 Arrive [time] at [airport code]
    🔄 [Direct/1 stop in CITY]
    ✅ Best for: [Economy/Speed/Comfort]

    ✈️ OPTION 2: [format as above]
    ✈️ OPTION 3: [format as above]

    🎯 BOOKING TIPS:
    • Best time to book: [specific advice]
    • Price trend: [when prices are typically lower]

    Use actual airlines that serve this route and realistic historical pricing."""

    # Generate flight options using LLM
    response = llm.invoke([SystemMessage(content=prompt)])
    state["result"] = response.content

    # Display flight results in styled format
    pretty_print(state["result"], "✈️ Top 3 Flight Options")

    # Create functional Google Flights booking link with proper date formatting
    def create_google_flights_link_with_dates(departure_city, destination, dates, travelers):
        """
        Generates a functional Google Flights URL with proper date formatting.
        Handles various date formats and converts them for Google Flights compatibility.
        """
        import urllib.parse

        # Smart date conversion for Google Flights URL format
        date_str = ""
        try:
            current_date = datetime.now()
            # Handle different date input formats
            if "next week" in dates.lower():
                next_week = current_date + timedelta(days=7)
                date_str = next_week.strftime("%Y-%m-%d")
            elif "july" in dates.lower():
                date_str = "2025-07-15"  # Default July date
            elif "december" in dates.lower():
                date_str = "2025-12-15"  # Default December date
            elif "august" in dates.lower():
                date_str = "2025-08-15"  # Default August date
            else:
                # Default to one month from now if cannot parse
                next_month = current_date + timedelta(days=30)
                date_str = next_month.strftime("%Y-%m-%d")
        except:
            # Ultimate fallback date
            date_str = "2025-08-15"

        # Get airport codes for more precise Google Flights search
        airport_prompt = f"""Convert these cities to their main airport codes:
        Departure: {departure_city}
        Destination: {destination}

        Return ONLY:
        FROM_CODE: [3-letter code]
        TO_CODE: [3-letter code]"""

        try:
            # Get airport codes from LLM
            airport_response = llm.invoke([SystemMessage(content=airport_prompt)])
            airport_text = airport_response.content.strip()

            from_code = ""
            to_code = ""

            # Parse airport codes from LLM response
            for line in airport_text.split('\n'):
                line = line.strip()
                if line.startswith('FROM_CODE:'):
                    from_code = line.replace('FROM_CODE:', '').strip()
                elif line.startswith('TO_CODE:'):
                    to_code = line.replace('TO_CODE:', '').strip()

            # Construct Google Flights URL with date parameters
            google_url = f"https://www.google.com/travel/flights?q=flights+from+{urllib.parse.quote(departure_city)}+to+{urllib.parse.quote(destination)}+on+{date_str}"

            return google_url, from_code, to_code

        except Exception as e:
            # Fallback URL construction if airport code lookup fails
            google_url = f"https://www.google.com/travel/flights?q=flights+from+{urllib.parse.quote(departure_city)}+to+{urllib.parse.quote(destination)}+{urllib.parse.quote(dates)}"
            return google_url, "", ""

    # Generate the booking link
    google_link, from_code, to_code = create_google_flights_link_with_dates(
        state['departure_city'],
        state['destination'],
        state['dates'],
        state['travelers']
    )

    # Display booking information with disclaimer
    booking_info = f"""🔗 BOOK YOUR FLIGHTS

✈️ Search live prices on Google Flights:
📅 Dates: {state['dates']}
👥 Travelers: {state['travelers']}
🌐 {google_link}

💡 Click above to search real flights and compare prices!

📝 NOTE: Flight data above is AI-generated for demo purposes."""

    pretty_print(booking_info, "🔗 Book Your Flights")

    # User satisfaction check with routing logic
    satisfied = get_input("Are you happy with these flight options? (yes/no):")

    if satisfied.lower() in ['yes', 'y', 'good', 'great', 'perfect', 'ok']:
        print("🎉 Great! Flight search completed successfully!")
        state["service_type"] = "done"  # This will end the session
    else:
        print("🔄 No problem! You can explore other services or try again.")
        # User will return to supervisor menu for other options

    return state

# +++++ SECTION 7: ITINERARY GENERATION ENGINE +++++
# This node creates personalized travel itineraries based on trip type and destination.
# It includes a feedback mechanism for iterative improvement and focuses on practical, actionable recommendations.
# The itineraries are concise but comprehensive, covering all essential travel planning aspects.

def create_itinerary(state: TravelState) -> TravelState:
    """
    Intelligent itinerary generator with trip type customization and user feedback loop.

    Capabilities:
    1. 6 different trip types with specialized recommendations
    2. Concise but comprehensive 3-day itinerary format
    3. Budget estimates and practical tips
    4. User feedback integration for itinerary improvement
    5. Wikipedia-style destination information integration
    """

    print("🗺️ Creating your personalized travel itinerary...")

    # Trip type selection for customized recommendations
    if not state.get("trip_type"):
        trip_menu = """🎯 Select your trip type for customized recommendations:

1. 💼 Business Trip - Meetings, networking, efficient transport
2. 🏖️ Leisure/Vacation - Relaxation, sightseeing, entertainment
3. 🎓 Educational/Cultural - Museums, history, local culture
4. 🏔️ Adventure/Outdoor - Activities, nature, sports
5. 👨‍👩‍👧‍👦 Family Trip - Kid-friendly, safe, educational
6. 💕 Romantic Getaway - Intimate, scenic, special experiences"""

        pretty_print(trip_menu, "🎯 Trip Type Selection", "red")
        choice = get_input("Choose your trip type (1-6):")

        # Map user choice to trip type
        trip_types = {
            '1': 'Business Trip', '2': 'Leisure/Vacation', '3': 'Educational/Cultural',
            '4': 'Adventure/Outdoor', '5': 'Family Trip', '6': 'Romantic Getaway'
        }

        state["trip_type"] = trip_types.get(choice, 'Leisure/Vacation')  # Default fallback
        print(f"✅ Perfect! Planning a {state['trip_type']} itinerary")

    print("📚 Gathering destination information and local insights...")

    # Optimized LLM prompt for concise, actionable itinerary
    prompt = f"""Create a CONCISE travel itinerary for:

    Destination: {state['destination']}
    Trip Type: {state['trip_type']}
    Travelers: {state['travelers']}
    Travel Dates: {state['dates']}

    Format (keep each section brief and actionable):

    🏛️ DESTINATION HIGHLIGHTS:
    • [2-3 key facts about {state['destination']}]
    • Best time to visit and weather info

    📅 3-DAY ITINERARY:
    Day 1: 🌅 [Main activity] - [Brief description with practical tip]
    Day 2: 🏛️ [Main activity] - [Brief description with practical tip]
    Day 3: 🍽️ [Main activity] - [Brief description with practical tip]

    🍽️ TOP 3 RESTAURANTS:
    • [Restaurant name] - [Cuisine type] 🍴 [Price range]
    • [Restaurant name] - [Cuisine type] 🥘 [Price range]
    • [Restaurant name] - [Cuisine type] 🍷 [Price range]

    💰 DAILY BUDGET ESTIMATE:
    • Accommodation: $[X-Y] per night
    • Meals: $[X-Y] per day
    • Activities: $[X-Y] per day
    • Transport: $[X-Y] per day

    🎯 {state['trip_type'].upper()} SPECIFIC TIPS:
    • [Tip 1 - specific to trip type]
    • [Tip 2 - specific to trip type]
    • [Tip 3 - specific to trip type]

    Keep it practical and use emojis for visual appeal!"""

    # Generate personalized itinerary
    response = llm.invoke([SystemMessage(content=prompt)])
    state["result"] = response.content

    # Display the generated itinerary
    pretty_print(state["result"], f"🗺️ {state['trip_type']} Itinerary for {state['destination']}")

    # User satisfaction check with improvement option
    satisfied = get_input("Are you happy with this itinerary? (yes/no):")

    if satisfied.lower() in ['yes', 'y', 'good', 'great', 'perfect', 'love', 'ok']:
        print("🎉 Excellent! Your itinerary is ready to use!")
        state["service_type"] = "done"  # End session successfully
    else:
        # Iterative improvement based on user feedback
        print("🔄 Let me improve it based on your preferences...")

        user_feedback = get_input("What would you like me to change? (e.g., more adventure, budget-friendly, cultural sites, food focus, etc.):")

        print("✨ Customizing your itinerary based on your feedback...")

        # LLM prompt for itinerary improvement
        improvement_prompt = f"""IMPROVE this itinerary based on specific user feedback:

        Original Destination: {state['destination']}
        Trip Type: {state['trip_type']}
        User Feedback: "{user_feedback}"

        ORIGINAL ITINERARY:
        {state['result']}

        Create an IMPROVED version that directly addresses their feedback:
        - Keep the same concise format
        - Focus on their specific requests
        - Maintain practical budget and timing information
        - Add relevant recommendations based on their preferences

        Make sure the improvements are clearly visible and address their concerns."""

        # Generate improved itinerary
        improved_response = llm.invoke([SystemMessage(content=improvement_prompt)])
        state["result"] = improved_response.content

        # Display improved version
        pretty_print(state["result"], f"🗺️ Improved {state['trip_type']} Itinerary")

        # Final satisfaction check
        final_check = get_input("How does this improved version look? (yes/no):")

        if final_check.lower() in ['yes', 'y', 'good', 'great', 'perfect', 'better', 'ok']:
            print("🎉 Perfect! Your customized itinerary is ready!")
            state["service_type"] = "done"
        else:
            print("🔄 Understood. You can always return for more adjustments!")

    # Disclaimer for AI-generated content
    print("\n📝 NOTE: This itinerary uses AI-generated recommendations. Verify details before travel.")

    return state

# +++++ SECTION 8: GRAPH ROUTING LOGIC +++++
# This section contains the routing functions that control the flow between different nodes in the LangGraph.
# These functions determine which node to visit next based on the current state and user choices.
# The routing logic is critical for maintaining proper conversation flow and preventing infinite loops.

def route_supervisor(state: TravelState) -> Literal["get_trip_info", "find_flights", "create_itinerary", "__end__"]:
    """
    Central routing function that determines the next node based on service_type.

    Routing Logic:
    - get_trip_info: Collect missing travel information
    - find_flights: Search and display flight options
    - create_itinerary: Generate travel itinerary
    - __end__: Terminate the conversation (when user selects "done")
    """
    service = state.get("service_type", "")

    if service == "get_trip_info":
        return "get_trip_info"
    elif service == "flights":
        return "find_flights"
    elif service == "itinerary":
        return "create_itinerary"
    else:  # service == "done" or any other value
        return "__end__"  # This terminates the graph execution

def route_back_to_supervisor(state: TravelState) -> Literal["supervisor"]:
    """
    Simple routing function that always returns to supervisor.
    Used by service nodes (flights, itinerary) to return to the main menu.
    """
    return "supervisor"

# +++++ SECTION 9: LANGGRAPH WORKFLOW CONSTRUCTION +++++
# This section builds the actual LangGraph workflow by connecting all nodes with appropriate edges.
# The graph structure defines how conversation flows between different components of the travel assistant.
# This is the architectural backbone that enables the multi-turn conversation capability.

def create_simple_graph():
    """
    Constructs the LangGraph workflow with all nodes and routing logic.

    Graph Structure:
    supervisor (entry point) → [get_trip_info, find_flights, create_itinerary, END]
    ↑                          ↓              ↓              ↓
    └──────────────────────────┴──────────────┴──────────────┘

    Flow Logic:
    1. Always starts at supervisor (welcome/menu)
    2. Supervisor routes to services based on user choice
    3. Services return to supervisor for additional options
    4. Only "done" choice terminates the conversation
    """

    # Initialize the StateGraph with our TravelState type
    workflow = StateGraph(TravelState)

    # Add all functional nodes to the graph
    workflow.add_node("supervisor", supervisor)           # Central controller
    workflow.add_node("get_trip_info", get_trip_info)    # Information collection
    workflow.add_node("find_flights", find_flights)      # Flight search
    workflow.add_node("create_itinerary", create_itinerary)  # Itinerary generation

    # Set supervisor as the entry point - all conversations start here
    workflow.set_entry_point("supervisor")

    # Define conditional routing from supervisor to services
    workflow.add_conditional_edges(
        "supervisor",           # Source node
        route_supervisor,       # Routing function
        {                       # Routing destinations
            "get_trip_info": "get_trip_info",
            "find_flights": "find_flights",
            "create_itinerary": "create_itinerary",
            "__end__": END      # Special END node terminates the graph
        }
    )

    # All service nodes return to supervisor for menu selection
    # This creates the loop that allows multiple services in one session
    workflow.add_edge("get_trip_info", "supervisor")
    workflow.add_edge("find_flights", "supervisor")
    workflow.add_edge("create_itinerary", "supervisor")

    # Compile the workflow into an executable graph
    return workflow.compile()

# +++++ SECTION 10: EXECUTION & UTILITY FUNCTIONS +++++
# This final section contains the main execution functions and utilities for running the ATLAS AI system.
# It includes graph visualization, the main run function, and comprehensive usage instructions.
# These functions provide the interface for users to interact with the travel assistant.

def show_graph():
    """
    Displays the LangGraph structure using visual diagram and text description.
    Useful for understanding the conversation flow and debugging the system architecture.
    """
    print("📊 ATLAS AI CONVERSATION FLOW:")

    try:
        # Generate and display visual graph using LangGraph's built-in visualization
        graph = create_simple_graph()
        graph_image = graph.get_graph().draw_mermaid_png()
        display(Image(graph_image))

        # Display system capabilities summary
        capabilities = """✅ System Features:
• Multilingual destination greetings using LLM
• Smart date parsing (converts "next week" to actual dates)
• Comprehensive missing information validation
• Top 3 flight options with Google Flights booking integration
• 6 specialized trip types with customized itineraries
• User feedback loop for itinerary improvements
• Beautiful HTML formatting for enhanced user experience
• Proper conversation flow with return-to-menu capability"""

        pretty_print(capabilities, "🚀 ATLAS AI Capabilities")

    except Exception as e:
        # Fallback display if graph visualization fails
        print(f"❌ Could not generate visual graph: {e}")
        print("📊 Text representation of conversation flow:")
        print("""
        START → supervisor (welcome & menu)
                    ↓
        ┌───────────┼───────────┐
        │           │           │
   get_trip_info  flights  itinerary
        │           │           │
        └───────────┼───────────┘
                    ↓
              supervisor (menu)
                    ↓
                   END
        """)
        print("\n📝 NOTE: This is an AI-powered demo system.")

def run_atlas():
    """
    Main execution function that initializes and runs the ATLAS AI travel assistant.
    Creates the initial state and starts the conversation flow.
    """

    # Display startup message
    startup_msg = """🚀 Initializing ATLAS AI Travel Assistant...
Ready to help you plan your perfect trip!"""
    pretty_print(startup_msg, "🎯 ATLAS AI Starting")

    # Create the executable graph
    graph = create_simple_graph()

    # Initialize the conversation state with empty values
    initial_state = TravelState(
        user_input="",              # User's raw travel description
        departure_city="",          # Parsed departure location
        destination="",             # Parsed destination
        dates="",                   # Converted travel dates
        travelers="",               # Number of travelers
        service_type="",            # Current service being accessed
        result="",                  # Generated results (flights/itinerary)
        trip_info_complete=False,   # Information collection status
        trip_type=""                # Selected trip type for itinerary
    )

    # Execute the graph - this starts the conversation and runs until completion
    graph.invoke(initial_state)

    # Display completion message
    completion_msg = "🎉 Thank you for using ATLAS AI! Have an amazing trip!"
    pretty_print(completion_msg, "✅ Session Complete")

# +++++ MAIN USAGE INSTRUCTIONS +++++
# Comprehensive usage guide for the ATLAS AI travel assistant system

if __name__ == "__main__":
    # Display comprehensive system information
    system_info = """🎯 ATLAS AI - Intelligent Travel Assistant

📋 QUICK START COMMANDS:
1️⃣ show_graph() - Display system architecture and flow
2️⃣ run_atlas() - Start your travel planning session

🚀 KEY FEATURES:
• Natural Language Processing - Understands "I want to go from NYC to Paris next week"
• Smart Date Conversion - Converts "next week" to actual calendar dates
• Multilingual Greetings - Welcomes you in your destination's language
• Flight Search - Top 3 options with direct Google Flights booking links
• Custom Itineraries - 6 trip types with personalized recommendations
• Feedback Loop - Improve itineraries based on your preferences
• Beautiful Interface - HTML-styled output for better readability

💡 EXAMPLE SESSION FLOW:
1. Start with: "I want to go from Boston to Tokyo next week"
2. Choose flights or itinerary planning
3. Get personalized recommendations
4. Book flights or save your itinerary
5. Provide feedback for improvements

⚠️ IMPORTANT NOTES:
• All flight prices and data are AI-generated for demonstration
• Always verify real prices and availability on booking sites
• This is a demo system showcasing LangGraph capabilities
• Requires OpenAI API key in Google Colab secrets"""

    pretty_print(system_info, "📖 ATLAS AI User Guide")

# Backward compatibility function
def show_atlas_graph():
    """Legacy function name - redirects to show_graph()"""
    show_graph()

In [ ]:
show_graph()

In [ ]:
run_atlas()